In [1]:
import cv2
import numpy as np

from src.utils import path_, env_, image, vars
path_.change_working_dir()

Current working dir: /home/younes/Desktop/projects/master


In [2]:
from src.datasets import DatasetChecker
import os
from natsort import natsorted
import matplotlib.pyplot as plt
from random import randint
from tqdm import tqdm

/home/younes/venv/master_/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
db_name = "ppdd"
db_checker_ = DatasetChecker(db_name)


Checking images files sanity...


Checking image files in datasets/splits/ppdd/train/images: 100%|██████████| 164264/164264 [00:00<00:00, 2531686.46it/s]
Checking image files in datasets/splits/ppdd/val/images: 100%|██████████| 20533/20533 [00:00<00:00, 2550014.63it/s]
Checking image files in datasets/splits/ppdd/test/images: 100%|██████████| 20533/20533 [00:00<00:00, 2536720.00it/s]

	All images are val

TRAIN report:
	3 dirs found.
		labels, annotations, images

VAL report:
	3 dirs found.
		labels, annotations, images

TEST report:
	3 dirs found.
		labels, annotations, images


In [4]:
db_split_path = os.path.join(env.get_split_path(db_name))
force = True
for set_name in vars.set_names:
    dir_name = "seg_masks"
    seg_masks_path = os.path.join(db_split_path, set_name, dir_name)
    path_.handle_path(seg_masks_path, force=force)

    labels_path = os.path.join(db_split_path, set_name, "labels")
    label_files = natsorted(os.listdir(labels_path))


    images_path = os.path.join(db_split_path, set_name, "images")
    sample_img = os.listdir(images_path)[0]
    image_type = "." + sample_img.split(".")[-1]

    loop = tqdm(label_files, total=len(label_files), desc="Creating masks for set: {}".format(set_name))
    if force:
        for label_file in loop:
            label_path = os.path.join(labels_path, label_file)
            file_name = ".".join((label_file.split(".")[:-1]))

            image_file = file_name + image_type
            image_path = os.path.join(images_path, image_file)

            mask, _ = image.label_to_mask(image_path, label_path)

            saving_path = os.path.join(seg_masks_path, file_name + ".png")
            cv2.imwrite(saving_path, mask)
    else:
        print("Force is not set skipping...")


Path is not found. Creating new path at: datasets/splits/ppdd/train/seg_masks


Creating masks for set: train: 100%|██████████| 164264/164264 [26:36<00:00, 102.90it/s]


Path is not found. Creating new path at: datasets/splits/ppdd/val/seg_masks


Creating masks for set: val: 100%|██████████| 20533/20533 [03:14<00:00, 105.34it/s]


Path is not found. Creating new path at: datasets/splits/ppdd/test/seg_masks


Creating masks for set: test: 100%|██████████| 20533/20533 [03:15<00:00, 105.15it/s]


In [5]:
split_dir = env.get_split_path(db_name)
labels_path = os.path.join(split_dir, "train", "labels")
images_path = os.path.join(split_dir, "train", "images")
rand_int = randint(0, len(os.listdir(images_path)) - 1)
# image_file = natsorted(os.listdir(images_path))[rand_int]
image_file = "00269_jpg.rf.0dc7743a8addc59d34013c445ea20d47.jpg"
image_path = os.path.join(images_path, image_file)
file_name = ".".join(image_file.split(".")[:-1])
label_path = os.path.join(labels_path, f"{file_name}.txt")

img = cv2.imread(image_path)
mask, annotated = image.label_to_mask(image_path, label_path)

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title("Image")
ax[0].axis("off")

colored_mask = image.visualize_mask(mask)
ax[1].imshow(colored_mask)
ax[1].set_title("Colorized Mask")
ax[1].axis("off")

ax[2].imshow(annotated)
ax[2].set_title("Annotated Image")
ax[2].axis("off")

plt.tight_layout()
plt.show()

[ WARN:0@1991.932] global loadsave.cpp:268 findDecoder imread_('datasets/splits/ppdd/train/images/00269_jpg.rf.0dc7743a8addc59d34013c445ea20d47.jpg'): can't open/read file: check file path/integrity
[ WARN:0@1991.932] global loadsave.cpp:268 findDecoder imread_('datasets/splits/ppdd/train/images/00269_jpg.rf.0dc7743a8addc59d34013c445ea20d47.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'copy'

In [12]:
from glob import glob

label_files = glob("datasets/splits/ppdd/train/labels/**/*.txt", recursive=True)

unique_ids = set()
for label_path in label_files:
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            try:
                raw_id = int(float(parts[0]))
                unique_ids.add(raw_id)
            except:
                print("Bad line:", line, "in", label_path)

print("Unique class IDs (before +1):", sorted(unique_ids))

Bad line: None 0.144653 0.444336 0.068604 0.569824 0.002466 0.674805 0.002466 0.899414 0.085815 0.900391 0.292969 0.900391 0.915039 0.902344 0.738281 0.667480 0.570312 0.444336 0.433838 0.444336 0.144653 0.444336
 in datasets/splits/ppdd/train/labels/181748.txt
Bad line: None 0.721191 0.698730 0.545898 0.241333 0.467773 0.242554 0.349854 0.700684 0.721191 0.698730
 in datasets/splits/ppdd/train/labels/173761.txt
Bad line: None 0.497070 0.244019 0.478516 0.303955 0.463623 0.350342 0.438721 0.430664 0.349365 0.707031 0.593750 0.708008 0.785156 0.707031 0.709473 0.522949 0.670410 0.429932 0.629395 0.328369 0.610352 0.284180 0.594727 0.244385 0.524414 0.244019 0.497070 0.244019
 in datasets/splits/ppdd/train/labels/161056.txt
Bad line: None 0.126953 0.463379 0.000099 0.659180 0.000099 0.919922 0.293213 0.918457 0.497314 0.918457 0.460449 0.737793 0.436768 0.618164 0.405273 0.463135 0.268066 0.464844 0.172363 0.463623 0.126953 0.463379
 in datasets/splits/ppdd/train/labels/225754.txt
Bad li